<a href="https://colab.research.google.com/github/jihoon99/bert_SQuAD/blob/master/2_BERT_embed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
goal : sentiment analysis

    1: positive(4->1로 추후 전처리)
    0: negative


이상한게 training set에는 neutral이 있음..

'''

'\ngoal : sentiment analysis\n\n    1: positive(4->1로 추후 전처리)\n    0: negative\n\n\n이상한게 training set에는 neutral이 있음..\n\n'

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 51kB 6.1MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=42b69afdd68e6f4c1c90da4644083c258307addfee40c001d87fa766f25fe6ad
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=8cc3dde92938928da7364204d249286ace6baa81e7a0bb3b6b82a31c94f2c6d8
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=0fdb075fa5f6f2dff75adf09862e72f8ad9686f3c93bcbf2cf4a6915aea286da
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 7.5MB/s 


In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import tensorflow_hub as hub # where to download models
from tensorflow.keras import layers
import bert

# Data Preprocess

## Load data

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]

data = pd.read_csv(
    "/content/drive/MyDrive/study/udemy_bert/train.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [ ]:
data.sentiment.unique()

array([0, 4])

## Preprocess

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text() #lxml포멧으로 변환하여 전처리
    # Removing the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Removing the URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Keeping only letters
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Removing additional whitespaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [ ]:
# data_clean = [clean_tweet(tweet) for tweet in data.text]

data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

In [ ]:
import pickle

# data clean을 저장함. 너무 오래걸림.
# with open('data_clean.pickle', 'wb') as f:
#     pickle.dump(data_clean, f, pickle.HIGHEST_PROTOCOL)

# load
with open('/content/drive/MyDrive/study/udemy_bert/data_clean.pickle', 'rb') as f:
    data_clean = pickle.load(f)

In [ ]:
data_clean[:4]

[" Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D",
 "is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!",
 ' I dived many times for the ball. Managed to save The rest go out of bounds',
 'my whole body feels itchy and like its on fire ']

## Tokenizer
- 토크나이징을 하기 위해선 버트 레이어를 만들어야함.

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False) # bert layer

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy() # vocab file을 numpy로 변환해서 갖고 있는다.
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy() # lower케이스를 할 경우.
tokenizer = FullTokenizer(vocab_file, do_lower_case) # 토크나이져

In [ ]:
# 예제
tokenizer.tokenize("My dog loves strawberries.")

['my', 'dog', 'loves', 'straw', '##berries', '.']

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("My dog loves strawberries."))

[2026, 3899, 7459, 13137, 20968, 1012]

In [ ]:
def encode_sentence(sent):
    ''' bert에 맞춰서 토큰들을 넣기'''
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [ ]:
# data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [ ]:
# with open('/content/drive/MyDrive/study/udemy_bert/data_inputs_embed.pickle', 'wb') as f:
#     pickle.dump(data_inputs, f, pickle.HIGHEST_PROTOCOL)

# load
with open('/content/drive/MyDrive/study/udemy_bert/data_inputs_embed.pickle', 'rb') as f:
    data_inputs = pickle.load(f)

In [ ]:
# data_labels = data.sentiment.values
# data_labels[data_labels == 4] = 1
# with open('/content/drive/MyDrive/study/udemy_bert/data_labels_embed.pickle', 'wb') as f:
#     pickle.dump(data_labels, f, pickle.HIGHEST_PROTOCOL)

# load
with open('/content/drive/MyDrive/study/udemy_bert/data_labels_embed.pickle', 'rb') as f:
    data_labels = pickle.load(f)

## DataSet transform
- 3개의 다른 인풋 센텐스를 만들어야 한다.

In [ ]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # turns 1 into 0 and vice versa
    return seg_ids

In [ ]:
# example

get_mask("[PAD]")

array(0)

In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)] # 문장, 답(ground_truth), 문장길이
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])


sorted_all = [([get_ids(sent_lab[0]), # word2idx
                get_mask(sent_lab[0]), # padding 정보 -> 패딩은 뒤에서 할 거다.
                get_segments(sent_lab[0])], # segment정보
               sent_lab[1]) # ground Truth 정보.
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [ ]:
# example

for sent_lab in data_with_len:
    print(sent_lab)
    break

[['[CLS]', '[SEP]'], 1, 2]


In [ ]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all, # generator로 불러오기
                                             output_types=(tf.int32, tf.int32))

In [ ]:
# example
sorted_all[0]

([[101, 5697, 2128, 5790, 2035, 2026, 2774, 102],
  array([1, 1, 1, 1, 1, 1, 1, 1]),
  [0, 0, 0, 0, 0, 0, 0, 0]],
 0)

In [ ]:
# example
next(iter(all_dataset))

(<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 101, 5697, 2128, 5790, 2035, 2026, 2774,  102],
        [   1,    1,    1,    1,    1,    1,    1,    1],
        [   0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>)

In [ ]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0)) # post padding이 정해지지 않음(None), 그리고 label은 ()에 저장

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset

# A = (tf.data.Dataset
#      .range(1, 5, output_type=tf.int32)
#      .map(lambda x: tf.fill([x], x)))

# B = A.padded_batch(10)

# for element in B.as_numpy_iterator():
#     print(element)

In [ ]:
NB_BATCHES = math.ceil(len(sorted_all)/BATCH_SIZE) # 배치 사이즈 구해주고
NB_BATCHES_TEST = NB_BATCHES // 10 # 테스트 사이즈 구해줌
all_batched.shuffle(NB_BATCHES) # shuffle

test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [ ]:
next(iter(all_dataset))

(<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 101, 5697, 2128, 5790, 2035, 2026, 2774,  102],
        [   1,    1,    1,    1,    1,    1,    1,    1],
        [   0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>)

# model

In [ ]:
# example
my_sent = ['[CLS]'] + tokenizer.tokenize('Rose are red.') + ['[SEP]']

# bert_layer는 3개를 인풋으로 받는다.
testing = bert_layer([tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32),0),  # 1. token id
            tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32),0),           # 2. mask_info
            tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32),0)])      # 3. segmant_info
testing

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-9.0243739e-01, -3.1239107e-01, -9.1837043e-01,  9.0993232e-01,
          7.9561085e-01, -2.2373390e-01,  9.0149468e-01,  3.3617881e-01,
         -7.0260376e-01, -9.9997807e-01, -6.1263007e-01,  9.0368819e-01,
          9.7990221e-01,  5.1885664e-01,  9.3426764e-01, -7.3485327e-01,
         -3.6272845e-01, -7.1974802e-01,  4.5090425e-01, -5.7366294e-01,
          7.2207272e-01,  9.9998593e-01, -1.6552918e-01,  3.1193751e-01,
          5.0942808e-01,  9.6602708e-01, -7.3075873e-01,  9.2318892e-01,
          9.6160442e-01,  7.4112266e-01, -6.9775683e-01,  2.1286364e-01,
         -9.9334133e-01, -1.9665204e-01, -9.6998727e-01, -9.9184394e-01,
          4.8266226e-01, -6.8694478e-01, -1.1749107e-01, -2.2487424e-03,
         -9.1652173e-01,  4.4211754e-01,  9.9997818e-01,  9.6349739e-02,
          6.1117959e-01, -3.0145600e-01, -1.0000000e+00,  5.5618232e-01,
         -8.9677763e-01,  8.4615594e-01,  8.6229694e-01,  6.3404870e-01,


In [ ]:
i,j = testing
i.shape, j.shape


(TensorShape([1, 768]), TensorShape([1, 6, 768]))

In [ ]:
# example
testing[0].shape, testing[1].shape

# 1,6,768 의 뜻은, 1은 배치사이즈, 6은 토큰길이(sequence length), 768은 embedding길이.

(TensorShape([1, 768]), TensorShape([1, 6, 768]))

In [ ]:
# vocab_size, embedding_dim is no more required
class DCNNBERTEmbedding(tf.keras.Model):
    
    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=False) # don't want to fine tune

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def embed_with_bert(self, all_tokens):
        _, embs = self.bert_layer([all_tokens[:, 0, :], # words
                                   all_tokens[:, 1, :], # mask
                                   all_tokens[:, 2, :]]) # segment
                                   # _  represents sentence [1,768]
                                   # embes represents [batch, sequences, embed_size]
        return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs) # [batch, sequences, embed_size]

        print(x.shape) # [None, None, 768]

        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# training

In [ ]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

In [ ]:
Dcnn = DCNNBERTEmbedding(nb_filters = NB_FILTERS,
                         FFN_units = FFN_UNITS,
                         nb_classes = NB_CLASSES,
                         dropout_rate = DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [ ]:
checkpoint_path = "/content/drive/MyDrive/study/udemy_bert/ckpoint/"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [ ]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

# result

In [ ]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])

Epoch 1/5
(None, None, 768)
(None, None, 768)
40623/40623 [==============================] - 2393s 58ms/step - loss: 0.3969 - accuracy: 0.8222
Checkpoint saved at /content/drive/MyDrive/study/udemy_bert/ckpoint/.
Epoch 2/5
40623/40623 [==============================] - 2376s 58ms/step - loss: 0.3745 - accuracy: 0.8344
Checkpoint saved at /content/drive/MyDrive/study/udemy_bert/ckpoint/.
Epoch 3/5
40623/40623 [==============================] - 2379s 58ms/step - loss: 0.3641 - accuracy: 0.8397
Checkpoint saved at /content/drive/MyDrive/study/udemy_bert/ckpoint/.
Epoch 4/5
25298/40623 [=================>............] - ETA: 11:41 - loss: 0.3349 - accuracy: 0.8555

# evaluation

In [ ]:
results = Dcnn.evaluate(test_dataset)
print(results)

In [ ]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)

    input_ids = get_ids(tokens)
    input_mask = get_mask(tokens)
    segment_ids = get_segments(tokens)

    inputs = tf.stack(
        [tf.cast(input_ids, dtype=tf.int32),
         tf.cast(input_mask, dtype=tf.int32),
         tf.cast(segment_ids, dtype=tf.int32)],
         axis=0)
    inputs = tf.expand_dims(inputs, 0) # simulates a batch

    output = Dcnn(inputs, training=False)

    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Output of the model: {}\nPredicted sentiment: negative".format(
            output))
    elif sentiment == 1:
        print("Output of the model: {}\nPredicted sentiment: positive".format(
            output))

In [ ]:
get_prediction("This actor is a deception.")